In [ ]:
#!/usr/bin/env python
# -*- encoding: utf-8 -*-
'''
@File    :   convert_dicom.ipynb
@Time    :   2025/05/08 10:49:07
@Author  :   Neutrin 
'''

# here put the import lib
import os 
import pandas as pd 
import pydicom as dicom
import matplotlib.pyplot as plt


In [6]:
#此函数基于Adam Jaamour开发的代码
#https://github.com/Adamouization/Breast-Cancer-Detection-Mammogram-Deep-Learning
def mammograms_retrieval():
    
    
    """
    此函数检索CBIS-DDSM中所有乳房X光照片(3103张)的路径。
    您需要从以下链接导入原始csv文件：
    "https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=22516629"。
    该函数输出4个csv文件。每个csv文件包含以下列：
    img id：对应于存放掩码的文件夹名称。
    img_path：图像在本地机器上的路径。
    label：图像病理学分类(良性或恶性)。BENIGN-WHITOUT-CALLBACK被替换为Benign。
    
    """
    #包含乳房X光照片的文件夹路径
    img_root = 'E:/Dataset/CBIS-DDSM/main_data/CBIS-DDSM'
    #包含csv文件的文件夹路径
    csv_root = 'E:/Dataset/CBIS-DDSM/main_data/csv'
    #您想保存包含乳房X光照片本地路径的新csv文件的文件夹
    csv_output_path = 'E:/Dataset/CBIS-DDSM/new_csv'
    folders = os.listdir(img_root)
    cases_dict = dict()  #保存图像ID和路径
    
    
    for f in folders:
        if f.endswith('_CC') or f.endswith('_MLO'):  #过滤掉裁剪过的图像
            path = list()

            for root, dirs, files in os.walk(img_root + '/' + f):  #检索图像的路径
                for d in dirs:
                    path.append(d)
                for filename in files:
                    path.append(filename)

            img_path = img_root + '/' + f + '/' + '/'.join(path)  #生成图像路径
            cases_dict[f] = img_path
            
    df = pd.DataFrame(list(cases_dict.items()), columns=['img', 'img_path']) 
    
    type_dict = {'Calc-Test': 'calc_case_description_test_set.csv',
                 'Calc-Training': 'calc_case_description_train_set.csv',
                 'Mass-Test': 'mass_case_description_test_set.csv',
                 'Mass-Training': 'mass_case_description_train_set.csv'}

    for t in type_dict.keys():  #根据类型处理图像
            df_subset = df[df['img'].str.startswith(t)]

            df_csv = pd.read_csv(csv_root + '/' + type_dict[t],
                                 usecols=['pathology', 'image file path'])  #读取原始csv文件
            df_csv['img'] = df_csv.apply(lambda row: row['image file path'].split('/')[0],
                                         axis=1)  #从路径中提取图像ID
            df_csv['pathology'] = df_csv.apply(
                lambda row: 'BENIGN' if row['pathology'] == 'BENIGN_WITHOUT_CALLBACK' else row['pathology'],
                axis=1)  #将病理学'BENIGN_WITHOUT_CALLBACK'替换为'BENIGN'

            df_csv = df_csv.drop_duplicates(subset=["img"]) #移除重复的乳房X光照片(原始csv文件将具有多个异常的乳房X光照片分配给不同的掩码)

            df_subset_new = pd.merge(df_subset, df_csv, how='inner',
                                     on='img') #基于图像ID合并图像路径和病理学信息

            df_subset_new = df_subset_new.drop(columns=["image file path"])

            df_subset_new.to_csv(csv_output_path + '/' + t.lower() + '.csv',
                                 index=False)  #以csv格式输出合并后的数据框

            print(t)
            print('data_cnt: %d' % len(df_subset_new))
            
    print('完成检索乳房X光照片路径！')

In [7]:
mammograms_retrieval()

Calc-Test
data_cnt: 284
Calc-Training
data_cnt: 1227
Mass-Test
data_cnt: 361
Mass-Training
data_cnt: 1231
完成检索乳房X光照片路径！
